In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/semi_project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
!pip install iterative-stratification

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.3 MB/s eta 0:00:00


In [5]:
df = pd.read_csv(f"{DATA_PATH}train_new_1.csv")
df.head()

,title,content,tag,date
0,버섯 대체육이 근육 형성에 미치는 영향,버섯으로 만든 햄버거 패티미국 영양학 Nutrition 잡지에 게재된 새 연구에 따...,비건 채식 비건디저트 대체육 채식식단 비건식단 비건식당 비건레시피 비건음식 비건간식,2023. 4. 12.
1,비건 레시피 콩고기보다 세이탄 어묵 대체육,대체육 좋아하시나요 사실 저는 대체육보다 채소나 콩 곡류 자체의 맛을 즐기는 편이라...,비건레시피 세이탄 대체육 집밥,2023. 3. 14.
2,식물성 햄 대체육 아이간식 추천하는 베러미트 밀레 쿠킹클래스 다녀왔어요,식물성 햄 대체육 아이간식 추천하는 베러미트 밀레 쿠킹클래스 다녀왔어요 작년 언젠,인류건강 동물복지 지구환경에 지구환경 베러미트 밀레코리아 쿠킹클래스 대체육 아이간식...,2023. 4. 13.
3,이게 고기가 아니라고 미래의 먹거리 대체육에 대한 모든 것,안녕하세요 더운 여름날 다들 잘 보내고 계신가요 캠핑도 가고 바베큐도 즐기고 계실...,IT TECH 대체육 지구온난화 온실가스 환경문제 동물복지 ESG 건강 환경 동물,2023. 7. 10.
4,대체육 관련주 TOP 5 feat 대장주,2020년 코로나가 덮쳤고 세상은 한동안 공포에 뒤덮있었습니다 그 뒤로 사람들은 안...,식물성고기관련주 식물성대체육 대체육시장,2023. 5. 23.


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24105 entries, 0 to 24104
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    24105 non-null  object
 1   content  24105 non-null  object
 2   tag      24105 non-null  object
 3   date     20325 non-null  object
dtypes: object(4)
memory usage: 753.4+ KB


In [7]:
model_name = "ainize/klue-bert-base-mrc"

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
from transformers import AutoModel
model = AutoModel.from_pretrained(model_name)
model

Some weights of BertModel were not initialized from the model checkpoint at ainize/klue-bert-base-mrc and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [10]:
train = df['title'] + ' ' + df['content']

In [11]:
question = '키워드가 무엇인가요?'
question_list = [question for i in range(len(train.tolist()))]
len(question_list)

24105

In [12]:
train = tokenizer(train.tolist(), question_list, max_length=512, padding='max_length', truncation=True)

In [13]:
input_ids = np.array(train['input_ids'])
token_type_ids = np.array(train['token_type_ids'])
attention_mask = np.array(train['attention_mask'])

input_ids.shape, token_type_ids.shape, attention_mask.shape

((24105, 512), (24105, 512), (24105, 512))

In [14]:
input = input_ids.copy()

In [15]:
# UNK 토큰으로 1인 값 0으로 치환
for i in range(input.shape[0]):
    input[i][input[i] == 1] = 0

In [16]:
def tostr_tag_token(tag):
    tag_list = [tag.split(' ') for tag in tag] # 문자열인 테그를 공백을 기준으로 나눔
    tag_token_str_list = [] # 테그 문자열을 담을 리스트 초기화
    for tags in tag_list:
        tag_token_list = [] # 각 text 마다 토큰화한 테그를 담을 리스트 초기화
        for tag in tags:
            tag_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tag))
            tag_token = ' '.join(map(str,tag_token)) # 토큰화한 테그를 문자열로 변환
            tag_token_list.append(tag_token)
        tag_token_list = list(filter(None, tag_token_list)) # 빈 문자열 제거
        tag_token_list = sorted(tag_token_list, key=lambda x: len(x), reverse=True) # 토큰화한 테그를 내림차순으로 정렬
        tag_token_str_list.append(tag_token_list)
    return tag_token_str_list

In [17]:
def target_data(train, tag, max_length=512):
    target = np.empty((0, max_length), int) # 빈 target 배열 생성
    for text_token, tag_token in tqdm(zip(train, tag), total=train.shape[0]):
        text_token = ' '.join(map(str, text_token)) # text를 각각 가져와 문자열로 변환
        # 각 문장에 해당하는 테그를 가져와 순서대로 해당하는 열을 1로 변환
        for token in tag_token:
            text_token = re.sub(f"{token}\s", '1 '*len(token.split(' ')), text_token)
        text_token = list(map(int, text_token.split(' '))) # 변환된 문자열을 공백을 기준으로 나눔
        text_token = np.array(text_token)
        text_token = (text_token == 1) + 0 # 1을 제외한 숫자를 0으로 치환
        target = np.append(target, text_token.reshape(1, -1), axis=0)
    return target

In [18]:
tag = tostr_tag_token(df['tag'])
target = target_data(input, tag)
target.shape

  0%|          | 0/24105 [00:00<?, ?it/s]

(24105, 512)

In [19]:
notag_list = []
for i in range(target.shape[0]):
    if not target[i].sum():
        notag_list.append(i)

In [20]:
len(notag_list)

2745

In [21]:
input_ids = np.delete(input_ids, notag_list, axis=0)
token_type_ids = np.delete(token_type_ids, notag_list, axis=0)
attention_mask  = np.delete(attention_mask , notag_list, axis=0)
target = np.delete(target, notag_list, axis=0)

input_ids.shape, token_type_ids.shape, attention_mask.shape, target.shape

((21360, 512), (21360, 512), (21360, 512), (21360, 512))

##### 데이터셋 나누기

In [22]:
def permutation_train_test_split(input_ids, token_type_ids, attention_mask, target, test_size=0.2, shuffle=True, random_state=SEED):
    test_num = int(input_ids.shape[0] * test_size)
    train_num = input_ids.shape[0] - test_num

    if shuffle:
        np.random.seed(random_state)
        shuffled = np.random.permutation(input_ids.shape[0])

        input_ids = input_ids[shuffled,:]
        token_type_ids = token_type_ids[shuffled,:]
        attention_mask = attention_mask[shuffled,:]
        target = target[shuffled,:]

        train_input_ids = input_ids[:train_num]
        train_token_type_ids = token_type_ids[:train_num]
        train_attention_mask = attention_mask[:train_num]

        test_input_ids = input_ids[train_num:]
        test_token_type_ids = token_type_ids[train_num:]
        test_attention_mask = attention_mask[train_num:]

        train_target = target[:train_num]

        test_target = target[train_num:]

    else:
        train_input_ids = input_ids[:train_num]
        train_token_type_ids = token_type_ids[:train_num]
        train_attention_mask = attention_mask[:train_num]

        test_input_ids = input_ids[train_num:]
        test_token_type_ids = token_type_ids[train_num:]
        test_attention_mask = attention_mask[train_num:]

        train_target = target[:train_num]

        test_target = target[train_num:]

    return train_input_ids, train_token_type_ids, train_attention_mask, test_input_ids, test_token_type_ids, test_attention_mask, train_target, test_target

In [23]:
train_input_ids, train_token_type_ids, train_attention_mask, test_input_ids, test_token_type_ids, test_attention_mask, train_target, test_target = permutation_train_test_split(input_ids, token_type_ids, attention_mask, target)

#### 학습데이터셋

In [24]:
train_input_ids.shape, train_token_type_ids.shape, train_attention_mask.shape, train_target.shape

((17088, 512), (17088, 512), (17088, 512), (17088, 512))

#### 테스트데이터셋

In [25]:
test_input_ids.shape, test_token_type_ids.shape, test_attention_mask.shape, test_target.shape

((4272, 512), (4272, 512), (4272, 512), (4272, 512))



---



In [26]:
class BlogDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, token_type_ids, y=None):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.y = y

    def __len__(self):
        return self.input_ids.shape[0]

    def __getitem__(self, idx):
        item = {}
        item['input_ids'] = torch.tensor(self.input_ids[idx])
        item['attention_mask'] = torch.tensor(self.attention_mask[idx])
        item['token_type_ids'] = torch.tensor(self.token_type_ids[idx])
        if self.y is not None:
            item['y'] = torch.Tensor(self.y[idx])
        return item

In [27]:
dt = BlogDataset(train_input_ids, train_attention_mask, train_token_type_ids, train_target)
dl = torch.utils.data.DataLoader(dt, batch_size=2)
batch = next(iter(dl))
batch

{'input_ids': tensor([[    2,  6872, 28714,  ...,     0,     0,     0],
         [    2,  1187,  2332,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'y': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 0., 0., 0.]])}

In [28]:
class Net(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.output_layer = torch.nn.Linear(self.model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        x = self.model(input_ids, attention_mask, token_type_ids)
        return self.output_layer(x[0])

In [29]:
model = Net(model_name)
pred = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
pred.shape

Some weights of BertModel were not initialized from the model checkpoint at ainize/klue-bert-base-mrc and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([2, 512, 1])

In [30]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        pred = model(batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['token_type_ids'].to(device))
        pred = pred.flatten(1)
        # pedding 한 부분 제외
        mask = batch['attention_mask'].eq(1).to(device)
        target = torch.masked_select(batch['y'].to(device), mask)
        result = torch.masked_select(pred, mask)
        loss = loss_fn(result.view(-1, 1), target.view(-1, 1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

In [31]:
@torch.no_grad()
def test_loop(dataloader, model, loss_fn, device):
    model.eval()
    sig = torch.nn.Sigmoid()
    pred_list = []
    epoch_loss = 0
    for batch in dataloader:
        pred = model(batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['token_type_ids'].to(device))
        pred = pred.flatten(1)

        # pedding 한 부분 제외
        if batch.get('y') is not None:
            mask = batch['attention_mask'].eq(1).to(device)
            target = torch.masked_select(batch['y'].to(device), mask)
            result = torch.masked_select(pred, mask)
            loss = loss_fn(result.view(-1, 1), target.view(-1, 1))
            epoch_loss += loss.item()

        pred = sig(pred)
        pred = pred.to('cpu').numpy()
        pred_list.append(pred)

    pred = np.concatenate(pred_list)
    epoch_loss /= len(dataloader)
    return epoch_loss, pred

In [32]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import gc

batch_size = 16
loss_fn = torch.nn.BCEWithLogitsLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 100
n_splits = 5
# cv = KFold(n_splits=n_splits, random_state=SEED, shuffle=True)

In [33]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
cv = MultilabelStratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)

In [34]:
is_holdout = False
reset_seeds(SEED)
best_f1_score_list = []
best_FP_avg_list = []
for i, (tri, vai) in enumerate(cv.split(train_input_ids, train_target)):
    if i <= 1:
        continue
    model = Net(model_name).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

    # 학습용
    train_dt = BlogDataset(train_input_ids[tri], train_attention_mask[tri], train_token_type_ids[tri], train_target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

    # 검증용
    valid_dt = BlogDataset(train_input_ids[vai], train_attention_mask[vai], train_token_type_ids[vai], train_target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    best_f1_score = 0
    best_fp_avg = 0
    patience = 0

    for epoch in tqdm(range(epochs)):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)
        pred = (pred > 0.5).astype(int)
        # 정답데이터에서 1인 값만 mask
        mask = train_target[vai] == 1
        f1 = f1_score(train_target[vai][mask], pred[mask], average='micro')
        # FP
        fp_score = ((pred == 1) & (train_target[vai] == 0)).sum()
        # 평군 FP
        fp_avg = fp_score/train_target[vai].shape[0]

        patience += 1
        print(train_loss, valid_loss, f1, fp_score, fp_avg, sep="\t")
        if f1 > best_f1_score:
            patience = 0
            best_f1_score = f1
            best_fp_avg = fp_avg  # F1 score가 가장 높을 때의 FP 점수 기록
            torch.save(model.state_dict(), f"model_{i}.pth")

        if patience == 3:
            break

    print(f"{i} 번째 폴드 best F1_score: {best_f1_score}")
    print(f"{i} 번째 폴드 best FP: {best_fp_avg}")
    best_f1_score_list.append(best_f1_score)
    best_FP_avg_list.append(best_fp_avg)

    del train_dl, train_dt, valid_dl, valid_dt
    gc.collect()

    del optimizer, model
    torch.cuda.empty_cache()

    if is_holdout:
        break
print(np.mean(best_f1_score_list))
print(np.mean(best_FP_avg_list))

Some weights of BertModel were not initialized from the model checkpoint at ainize/klue-bert-base-mrc and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/857 [00:00<?, ?it/s]

0.09557470354135283	0.08098862306127008	0.7591276841057579	35143	10.403493191237418


  0%|          | 0/857 [00:00<?, ?it/s]

...

0.07370839278377393	0.07909902231324287	0.74037590133575	28457	8.497163332338012


  0%|          | 0/859 [00:00<?, ?it/s]

0.06091220454422364	0.08429175418402468	0.7449958627211474	27547	8.225440429979098


  0%|          | 0/859 [00:00<?, ?it/s]

0.04822635736069246	0.09064181039021128	0.763396902951259	31633	9.445506121230219
4 번째 폴드 best F1_score: 0.8033117932148627
4 번째 폴드 best FP: 11.964168408480143
0.8002589995473248
11.193316164379645


#### 추론하기

In [35]:
test_dt = BlogDataset(test_input_ids, test_attention_mask, test_token_type_ids)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=batch_size, shuffle = False)

pred_list = []
for i in range(n_splits):
    model = Net(model_name).to(device)
    state_dict = torch.load(f"model_{i}.pth") # 가중치 불러오기
    model.load_state_dict(state_dict) # 모델에 가중치 세팅

    _, pred = test_loop(test_dl, model, loss_fn, device)
    pred_list.append(pred)

pred = np.mean(pred_list, axis=0)
pred = (pred > 0.5).astype(int)

Some weights of BertModel were not initialized from the model checkpoint at ainize/klue-bert-base-mrc and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: ignored

#### f1_score, FP 계산

In [ ]:
mask = test_target == 1
f1 = f1_score(test_target[mask], pred[mask], average='micro')
fp_score = ((pred == 1) & (test_target == 0)).sum()
fp_avg = fp_score / test_target.shape[0]
print(f"F1_SCORE : {f1}")
print(f"FP_AVG : {fp_avg}")

#### 추론한 토큰 역 토큰화

In [ ]:
def convert_to_tokens(tokenizer, test_input_ids, pred):
    tag_tokens = []
    for i in range(len(pred)): # pred 값이 1인 부분만 추출
        tag_token = tokenizer.convert_ids_to_tokens(test_input_ids[i][pred[i] == 1])
        tag_tokens.append(tag_token)
    return tag_tokens

In [ ]:
# 후에 토큰들 연결 시키는 작업 필요
tag_tokens = convert_to_tokens(tokenizer, test_input_ids, pred)

In [ ]:
tag_tokens[1]